# Trend of guider drifts
Craig Lage - 05-Jan-26

In [ ]:
import pandas as pd
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.obs.lsst import LsstCam
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors


In [ ]:
dayObs = 20251214
guiderTable = pd.read_json(f'/project/rubintv/LSSTCam/guiders/sidecar_metadata/dayObs_{dayObs}.json').T
guiderTable = guiderTable.sort_index()

In [ ]:
guiderTable['Az drift (arcsec total)']

In [ ]:
guiderTable.iloc[369]['Az drift (arcsec total)']

In [ ]:
guiderTable.columns

## Get the guider drift data from RubinTV and calculate the ptg drift

In [ ]:
startDay = 20251115
endDay = 20260111
az_drifts = []
az_stds = []
el_drifts = []
total_drifts = []
total_stds = []
expIds = []
dayObss = []
dayObs = startDay
while dayObs <= endDay:
        try:
            guiderTable = pd.read_json(f'/project/rubintv/LSSTCam/guiders/sidecar_metadata/dayObs_{dayObs}.json').T
            guiderTable = guiderTable.sort_index()
            print(dayObs, len(guiderTable))
            these_az_drifts = []
            these_el_drifts = []
            these_total_drifts = []
            for i in range(1, len(guiderTable)+1):
                try:
                    seqNum = int(guiderTable.index[i])
                    thisGuider = guiderTable[guiderTable.index == seqNum]
                    expId = int(dayObs * 1.0E5 + seqNum)
                    az_drift = float(thisGuider['Az drift (arcsec total)'])
                    el_drift = float(thisGuider['Alt drift (arcsec total)'])
                    if dayObs < 20251121:
                        az_drift *= 30.0#expTime
                        el_drift *= 30.0#expTime
                    total_drift = np.sqrt(az_drift**2 + el_drift**2)
                    data = np.array([az_drift, el_drift])
                    if np.isnan(data).any():
                        #print(f"{expId} had nans!")
                        continue
                    #total_drifts.append(total_drift)
                    #expIds.append(expId)
                    these_az_drifts.append(abs(az_drift))
                    these_el_drifts.append(abs(el_drift))
                    these_total_drifts.append(total_drift)
                    #print(f"{expId} succeeded!")
                except Exception as e:
                    #print(f"Failed with error: {e}")
                    #print(f"{expId} failed!")
                    continue
            dayObss.append(dayObs)
            print(dayObs, np.nanmedian(these_az_drifts), np.nanmedian(these_el_drifts))
            az_drifts.append(np.nanmedian(these_az_drifts))
            az_stds.append(np.nanstd(these_az_drifts))
            el_drifts.append(np.nanmedian(these_el_drifts))
            total_drifts.append(np.nanmean(these_total_drifts))
            total_stds.append(np.nanstd(these_total_drifts))
        except:
            print(f"{dayObs} failed")
            dayObs = calcNextDay(dayObs)
            continue
        print(f" Done with {dayObs}")
        dayObs = calcNextDay(dayObs)     
#filename = f"/home/cslage/DATA/guider_drifts_total_{startDay}_{endDay}.pkl"
#with open(filename, 'wb') as f:
#    pkl.dump([expIds, azs, els, az_drifts, el_drifts, total_drifts, ptg_az_drifts, ptg_el_drifts, ptg_total_drifts], f)
    

In [ ]:
print(len(az_drifts), len(el_drifts), len(dayObss), len(total_drifts))

In [ ]:
for i, dayObs in enumerate(dayObss):
    print(i, dayObs, total_drifts[i])

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,5))
plt.suptitle(f"Guider total drift trend {startDay}-{endDay}", fontsize=18)
xaxis = list(range(len(dayObss)))
med1 = np.nanmedian(total_drifts[0:26])
ax.text(4, 0.4, f"Median = {med1:.2f}")
ax.text(30, 0.4, f"Median = {med2:.2f}")
med2 = np.nanmedian(total_drifts[26:-1])
print(med1, med2)
#ax.errorbar(xaxis, total_drifts, yerr=np.array(total_stds)/30)
ax.scatter(xaxis, total_drifts)
#axs[0].errorbar(xaxis, az_drifts, yerr=az_stds)
ax.set_xticks(xaxis)
ax.set_xticklabels(dayObss, rotation=90)
ax.set_ylim(0, 0.50)
ax.axvline(26, ls='--', color='black')
ax.set_ylabel("Mean total guider drift (arcseconds/exposure)")
fig.savefig(f"/home/cslage/DATA/Guider_drifts_total_trend_{startDay}_{endDay}.png", bbox_inches='tight', pad_inches=1.2)

In [ ]:
fig, axs = plt.subplots(2,1)
xaxis = list(range(len(dayObss)))
axs[0].scatter(xaxis, az_drifts)
#axs[0].errorbar(xaxis, az_drifts, yerr=az_stds)
axs[0].set_xticks([])
axs[0].set_ylim(0,0.5)
axs[1].scatter(xaxis, el_drifts)
axs[1].set_ylim(0,0.5)
axs[1].set_xticks(xaxis)
axs[1].set_xticklabels(dayObss, rotation=90)


In [ ]:
startDay = 20251026
endDay = 20251027

filename = f"/home/cslage/DATA/guider_drifts_total_{startDay}_{endDay}.pkl"
with open(filename, 'rb') as f:
    [expIds, azs, els, az_drifts, el_drifts, total_drifts, ptg_az_drifts, ptg_el_drifts, ptg_total_drifts] = pkl.load(f)


In [ ]:
plt.figure(figsize=(12,5))
plt.suptitle(f"Guider drifts vs Pointing Error drifts {startDay}-{endDay}")
plt.subplot(1,2,1)
plt.title("Azimuth drift/exposure")
plt.plot(np.array(ptg_az_drifts), marker='x', ms=0.1, label="Ptg error")
plt.plot(az_drifts, marker='o', ms=0.1, alpha=0.5, label="Guider")
plt.axvline(227, ls='--', color='black')
plt.text(50, 0.7, f"{startDay}", color='black')
plt.text(250, 0.7, f"{endDay}", color='black')
plt.xlabel("Exposure list index")
plt.ylabel("Drift/exposure (arcsec)")
plt.ylim(-0.8, 0.8)
plt.legend(loc = 'lower left')
plt.subplot(1,2,2)
plt.title("Elevation drift/exposure")
plt.plot(np.array(ptg_el_drifts), marker='x', ms=0.1)
plt.plot(el_drifts, marker='o', ms=0.1, alpha=0.5)
plt.axvline(227, ls='--', color='black')
plt.text(50, 0.7, f"{startDay}", color='black')
plt.text(250, 0.7, f"{endDay}", color='black')
plt.xlabel("Exposure list index")
plt.ylabel("Drift/exposure (arcsec)")
plt.ylim(-0.8, 0.8)
plt.savefig(f"/home/cslage/DATA/Guider_Drifts_Pointing_EFD_{startDay}_{endDay}.png")

In [ ]:
[azDrift, elDrift] = CalculateDrift(2025110300052, printOut=True)
print(azDrift, elDrift)